In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

# Leer los datos de archivo csvL
url =  "C:/Users/PC/Documents/GitHub/GitFlow-en-Github/ML003/PIA/Practica PIA/entrenamiento.csv"
df_entrenamiento = pd.read_csv(url)

# Leer los datos de archivo csv
url =  "C:/Users/PC/Documents/GitHub/GitFlow-en-Github/ML003/PIA/Practica PIA/prueba.csv"
df_prueba = pd.read_csv(url)

# Separar características y objetivo en el conjunto de entrenamiento
#X = df_entrenamiento.drop(columns=['id', 'engagement'])

X = df_entrenamiento.drop(columns=['id', 'title_word_count','engagement','speaker_speed','silent_period_rate','easiness'])
y = df_entrenamiento['engagement'].astype(int)

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir StratifiedKFold
cv = StratifiedKFold(n_splits=5)

# Definir el rango de parámetros para KNeighborsClassifier
param_grid_knn = {
    'classifier__n_neighbors': list(range(1, 31)),  # Explorar valores de 1 a 30 para n_neighbors
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']
}

# Crear pipeline con StandardScaler y KNeighborsClassifier
pipeline_knn = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', KNeighborsClassifier())
])

# Realizar la búsqueda de hiperparámetros con GridSearchCV
grid_search_knn = GridSearchCV(pipeline_knn, param_grid_knn, cv=cv, scoring='roc_auc', n_jobs=-1)
grid_search_knn.fit(X_train, y_train)

# Obtener el mejor modelo y los mejores parámetros
mejor_modelo_knn = grid_search_knn.best_estimator_
mejores_params_knn = grid_search_knn.best_params_

# Evaluar el mejor modelo en el conjunto de validación
val_predictions_knn = mejor_modelo_knn.predict_proba(X_val)[:, 1]
val_roc_auc_knn = roc_auc_score(y_val, val_predictions_knn)
val_accuracy_knn = accuracy_score(y_val, mejor_modelo_knn.predict(X_val))

# Mostrar resultados
print(f"Mejores Parámetros KNN: {mejores_params_knn}")
print(f"Validation ROC AUC KNN: {val_roc_auc_knn}")
print(f"Validation Accuracy KNN: {val_accuracy_knn}")



Mejores Parámetros KNN: {'classifier__metric': 'manhattan', 'classifier__n_neighbors': 30, 'classifier__weights': 'distance'}
Validation ROC AUC KNN: 0.8709092187139121
Validation Accuracy KNN: 0.9242424242424242
